<center><h1>Deep Learning for Stock Price Direction:</h1></center>
<p>Binary Classification: Predicting if the stock will go up or down based on the past 5 days</p>

<h4>Imports:</h4>

In [266]:
#Imports for Stock Price data download
from pandas_datareader import data as pdr 
import fix_yahoo_finance as yf

#Imports for data manipulation
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

#Imports for Deep Learning Model 
from keras.models import Sequential 
from keras.layers import LSTM,Dense

<h4>Download stock ticker data and obtain the closing prices:</h4>

In [314]:
#Download Stock Price Data
yf.pdr_override()
ticker = "NCR"
data = pdr.get_data_yahoo(ticker,start="1970-01-01",end="2017-11-05")
data = data["Close"]
prices = data.values

[*********************100%***********************]  1 of 1 downloaded

<h4>Create vectors of sliding 5 day windows and try to predict the direction of the market on the 6th day</h4>

In [255]:
#Create 5 day price with direction
five_day_prices = []
direction = []
for i in range(data.shape[0]-5): 
    sub_prices = prices[i:i+5]
    five_day_prices.append(sub_prices)
    if prices[i+5] <= sub_prices[0]: 
        direction.append(0)
    else: 
        direction.append(1)
five_day_prices = np.array(five_day_prices)
direction = np.array(direction)

<h4>Reshape our inputs and targets for use of the LSTM model</h4>

In [267]:
five_day_prices_reshaped = five_day_prices.reshape(five_day_prices.shape[0],1,5)
direction_reshaped = np.reshape(direction,(direction.shape[0],1,1))

<h4>Create Train/Test Split of data:</h4>

In [268]:
X_train, X_test, y_train, y_test = train_test_split(five_day_prices_reshaped, direction_reshaped, test_size=0.20, random_state=42)

<h4>Create an LSTM deep learning model:</h4>

In [272]:
def create_model(): 
    model = Sequential()
    model.add(LSTM(10,input_shape=(1,5),return_sequences=True))
    model.add(Dense(5,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

<h4>Instantiate and train our model:</h4>

In [273]:
model = create_model()
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Train on 4204 samples, validate on 1052 samples
Epoch 1/20
4204/4204 [==============================] - 3s - loss: 0.6713 - acc: 0.6644 - val_loss: 0.6555 - val_acc: 0.7063

<h4>Save the model:</h4>

In [275]:
model.save('NCR_LSTM.h5')

<h4>Predict on the model:</h4>

In [312]:
preds = model.predict_classes(X_test).flatten()

 800/1052 [=====================>........] - ETA: 0s

<h4>Performance:</h4>

In [313]:
count_arr = preds == y_test.flatten()
count = len(count_arr[count_arr == True])
print("Correct Predictions: ",count,"\nIncorrect Predictions: ",len(count_arr) - count)
print("Test Accuracy: ",count/len(count_arr)*100)

Correct Predictions:  875 
Incorrect Predictions:  177
Test Accuracy:  83.17490494296578
